In [1]:
!py --version

Python 3.10.11


In [2]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

2.6.0+cu118
CUDA available: True
GPU: NVIDIA GeForce RTX 2050


## Import Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.models import TimeXer, LSTM, TimeLLM
from neuralforecast.losses.pytorch import MSE, MAE, RMSE, QuantileLoss 

## Load Data

In [2]:
data_raw = pd.read_csv("Kolkata-daily.csv",  parse_dates=['time', 'sunrise (iso8601)', 'sunset (iso8601)'])
data_raw.head()

,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),rain_sum (mm),sunshine_duration (s),precipitation_hours (h),dew_point_2m_mean (°C),winddirection_10m_dominant (°),wind_speed_10m_max (km/h),sunrise (iso8601),sunset (iso8601)
0,1975-01-01,19.3,25.9,13.3,0.0,35364.91,0,9.6,64,9.4,1975-01-01 06:16:00,1975-01-01 17:02:00
1,1975-01-02,20.3,27.0,12.6,0.0,35346.28,0,13.0,150,8.4,1975-01-02 06:16:00,1975-01-02 17:03:00
2,1975-01-03,20.0,25.1,15.9,0.0,29732.62,0,12.7,11,10.1,1975-01-03 06:17:00,1975-01-03 17:03:00
3,1975-01-04,19.6,25.6,14.8,0.0,35341.67,0,11.1,354,14.4,1975-01-04 06:17:00,1975-01-04 17:04:00
4,1975-01-05,18.0,24.0,12.8,0.0,35385.59,0,8.0,339,12.8,1975-01-05 06:17:00,1975-01-05 17:05:00


## Format Data to fit Neuralforecast

In [3]:
data = data_raw.copy()
#data.rename(columns={'time': 'ds'}, inplace=True)
#data.rename(columns={'rain_sum (mm)': 'y'}, inplace=True)
data['sunrise (iso8601)'] = data['sunrise (iso8601)'].dt.time
data['sunset (iso8601)'] = data['sunset (iso8601)'].dt.time
data['sunrise'] = data['sunrise (iso8601)'].apply(lambda t: t.hour * 3600 + t.minute * 60 + t.second)
data['sunset'] = data['sunset (iso8601)'].apply(lambda t: t.hour * 3600 + t.minute * 60 + t.second)
data.drop(columns=['sunrise (iso8601)', 'sunset (iso8601)'], inplace=True)
data['unique_id'] = 1.0
data

,time,temperature_2m_mean (°C),temperature_2m_max (°C),temperature_2m_min (°C),rain_sum (mm),sunshine_duration (s),precipitation_hours (h),dew_point_2m_mean (°C),winddirection_10m_dominant (°),wind_speed_10m_max (km/h),sunrise,sunset,unique_id
0,1975-01-01,19.3,25.9,13.3,0.0,35364.91,0,9.6,64,9.4,22560,61320,1.0
1,1975-01-02,20.3,27.0,12.6,0.0,35346.28,0,13.0,150,8.4,22560,61380,1.0
2,1975-01-03,20.0,25.1,15.9,0.0,29732.62,0,12.7,11,10.1,22620,61380,1.0
3,1975-01-04,19.6,25.6,14.8,0.0,35341.67,0,11.1,354,14.4,22620,61440,1.0
4,1975-01-05,18.0,24.0,12.8,0.0,35385.59,0,8.0,339,12.8,22620,61500,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18356,2025-04-04,30.2,34.2,26.1,0.0,13585.84,0,21.8,226,10.6,19560,64320,1.0
18357,2025-04-05,31.8,38.5,26.2,0.0,41434.78,0,18.9,227,14.3,19500,64380,1.0
18358,2025-04-06,31.0,37.8,26.3,1.0,39600.00,2,22.3,182,17.9,19440,64380,1.0
18359,2025-04-07,30.2,36.8,25.6,0.3,37035.58,1,21.7,161,15.3,19380,64380,1.0


In [4]:
test_size = data.index[data['time'] == '2020-12-31']
data_train = data.loc[data.index <= test_size[0]]
data_test = data.loc[data.index > test_size[0]]

In [5]:
print(f"Training from {data_train['time'].min().date()} to {data_train['time'].max().date()}, size = {len(data_train)}")
print(f"Testing from {data_test['time'].min().date()} to {data_test['time'].max().date()}, size = {len(data_test)}")

Training from 1975-01-01 to 2020-12-31, size = 16802
Testing from 2021-01-01 to 2025-04-08, size = 1559


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18361 entries, 0 to 18360
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   time                            18361 non-null  datetime64[ns]
 1   temperature_2m_mean (°C)        18361 non-null  float64       
 2   temperature_2m_max (°C)         18361 non-null  float64       
 3   temperature_2m_min (°C)         18361 non-null  float64       
 4   rain_sum (mm)                   18361 non-null  float64       
 5   sunshine_duration (s)           18361 non-null  float64       
 6   precipitation_hours (h)         18361 non-null  int64         
 7   dew_point_2m_mean (°C)          18361 non-null  float64       
 8   winddirection_10m_dominant (°)  18361 non-null  int64         
 9   wind_speed_10m_max (km/h)       18361 non-null  float64       
 10  sunrise                         18361 non-null  int64         
 11  su

In [7]:
data.to_csv('Kolkata-daily-nfinput.csv')

## Training TimeXer Model

In [ ]:
model_transformer1 = TimeXer(h=7,
                input_size=30,
                n_series=1,
                batch_size=128,
                patch_len=12,
                hidden_size=128,
                n_heads=16,
                e_layers=2,
                d_ff=256,
                factor=1,
                dropout=0.2,
                use_norm=True,
                loss=MSE(),
                learning_rate=0.0001,
                valid_loss=MAE(),
                early_stop_patience_steps=10,
                max_steps=2000)
nf_transformer_min = NeuralForecast(models=[model_transformer1], freq='D')
nf_transformer_max = NeuralForecast(models=[model_transformer1], freq='D')
nf_transformer_max.fit(df=data_train, val_size=365, time_col='time', target_col='temperature_2m_max (°C)', id_col='unique_id')
nf_transformer_min.fit(df=data_train, val_size=365, time_col='time', target_col='temperature_2m_min (°C)', id_col='unique_id')

Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 2050') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                   | Params | Mode 
----------------------------------------------------------------
0 | loss         | MSE                    | 0      | train
1 | valid_loss   | MAE                    | 0      | train
2 | padder_train | ConstantPad1d          | 0      | train
3 | scaler       | TemporalNorm           | 0      | train
4 | en_embedding | EnEmbedding            | 1.7 K  | train
5 | ex_embedding | DataEmbedding_inverted | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=2000` reached.


In [11]:
nf_transformer_max.save(path='./models/model_max/', overwrite=True, save_dataset=True)
nf_transformer_min.save(path='./models/model_min/', overwrite=True, save_dataset=True)

## Forecasting with TimeXer

In [7]:
rolling_forecasts_max_timexer = []

for i in range(len(data_test)): 
    predict_date = data_test.iloc[i]['time']
    
    # Find end index for the window ending just before predict_date
    try:
        end_i = data.index[data['time'] == predict_date][0]
    except IndexError:
        print(f"Date {predict_date} not found in `data`, skipping.")
        continue

    input_window = data.iloc[end_i - 30 : end_i]

    preds = nf_transformer_max.predict(df=input_window, verbose=False)

    # Preparing one record with 7 predicted values
    record = {'input_until': predict_date-pd.Timedelta(days=1)}
    for j in range(7):
        record[f'pred_day{j+1}'] = preds.iloc[j]['TimeXer']

    rolling_forecasts_max_timexer.append(record)

# Convert to DataFrame
forecast_df_max_timexer = pd.DataFrame(rolling_forecasts_max_timexer)
forecast_df_max_timexer

NameError: name 'nf_transformer_max' is not defined

In [ ]:
rolling_forecasts_min_timexer = []

for i in range(len(data_test)):  
    predict_date = data_test.iloc[i]['time']
    
    # Find end index for the window ending just before predict_date
    try:
        end_i = data.index[data['time'] == predict_date][0]
    except IndexError:
        print(f"Date {predict_date} not found in `data`, skipping.")
        continue

    input_window = data.iloc[end_i - 30 : end_i]

    preds = nf_transformer_min.predict(df=input_window, verbose=False)

    # Preparing one record with 7 predicted values
    record = {'input_until': predict_date-pd.Timedelta(days=1)}
    for j in range(7):
        record[f'pred_day{j+1}'] = preds.iloc[j]['TimeXer']

    rolling_forecasts_min_timexer.append(record)

# Convert to DataFrame
forecast_df_min_timexer = pd.DataFrame(rolling_forecasts_min_timexer)
forecast_df_min_timexer

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

,input_until,pred_day1,pred_day2,pred_day3,pred_day4,pred_day5,pred_day6,pred_day7
0,2020-12-31,13.137679,13.088698,12.985536,13.184299,13.149577,13.071213,12.807915
1,2021-01-01,13.034383,12.943583,12.901439,13.070148,13.003500,12.906763,12.617537
2,2021-01-02,12.500472,12.496919,12.528550,12.693856,12.754368,12.607304,12.428665
3,2021-01-03,12.489101,12.431701,12.506416,12.674364,12.757535,12.639399,12.500160
4,2021-01-04,13.055624,12.854541,12.856647,12.911273,13.059003,12.925626,12.795364
...,...,...,...,...,...,...,...,...
1554,2025-04-03,25.596386,25.663139,25.722109,25.764709,26.236231,25.853722,26.044931
1555,2025-04-04,25.879656,25.907093,26.036579,26.063267,26.594130,26.133013,26.252419
1556,2025-04-05,26.022058,26.056139,26.194601,26.107693,26.681789,26.262943,26.383354
1557,2025-04-06,26.124247,26.149715,26.207762,26.187883,26.662809,26.263128,26.371281


In [ ]:
timexer_pred_1 = pd.concat((forecast_df_max_timexer.iloc[:]['pred_day1'], forecast_df_min_timexer.iloc[:]['pred_day1']), axis=1)
timexer_pred_1.columns = ['max', 'min']
timexer_pred_2 = pd.concat((forecast_df_max_timexer.iloc[:-1]['pred_day2'], forecast_df_min_timexer.iloc[:-1]['pred_day2']), axis=1)
timexer_pred_2.columns = ['max', 'min']
timexer_pred_3 = pd.concat((forecast_df_max_timexer.iloc[:-2]['pred_day3'], forecast_df_min_timexer.iloc[:-2]['pred_day3']), axis=1)
timexer_pred_3.columns = ['max', 'min']
timexer_pred_4 = pd.concat((forecast_df_max_timexer.iloc[:-3]['pred_day4'], forecast_df_min_timexer.iloc[:-3]['pred_day4']), axis=1)
timexer_pred_4.columns = ['max', 'min']
timexer_pred_5 = pd.concat((forecast_df_max_timexer.iloc[:-4]['pred_day5'], forecast_df_min_timexer.iloc[:-4]['pred_day5']), axis=1)
timexer_pred_5.columns = ['max', 'min']
timexer_pred_6 = pd.concat((forecast_df_max_timexer.iloc[:-5]['pred_day6'], forecast_df_min_timexer.iloc[:-5]['pred_day6']), axis=1)
timexer_pred_6.columns = ['max', 'min']
timexer_pred_7 = pd.concat((forecast_df_max_timexer.iloc[:-6]['pred_day7'], forecast_df_min_timexer.iloc[:-6]['pred_day7']), axis=1)
timexer_pred_7.columns = ['max', 'min']

## Evaluation of TimeXer

In [14]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

In [15]:
rsme_timexer_1 = root_mean_squared_error(data_test[['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_1)
mae_timexer_1 = mean_absolute_error(data_test[['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_1)
mape_timexer_1 = mean_absolute_percentage_error(data_test[['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_1)
r2_timexer_1 = r2_score(data_test[['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_1)
print(f"For 1 day future -> RMSE: {rsme_timexer_1}, MAE: {mae_timexer_1}, MAPE: {mape_timexer_1}, R2: {r2_timexer_1}")

For 1 day future -> RMSE: 1.3096089628081293, MAE: 0.9873336444534194, MAPE: 0.040737305935170456, R2: 0.9071454048622727


In [16]:
rsme_timexer_2 = root_mean_squared_error(data_test.iloc[1:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_2)
mae_timexer_2 = mean_absolute_error(data_test.iloc[1:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_2)
mape_timexer_2 = mean_absolute_percentage_error(data_test.iloc[1:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_2)
r2_timexer_2 = r2_score(data_test.iloc[1:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_2)
print(f"For 2 day future -> RMSE: {rsme_timexer_2}, MAE: {mae_timexer_2}, MAPE: {mape_timexer_2}, R2: {r2_timexer_2}")

For 2 day future -> RMSE: 1.6998872988244869, MAE: 1.2923696975805945, MAPE: 0.05378680207161202, R2: 0.8436666424407724


In [17]:
rsme_timexer_3 = root_mean_squared_error(data_test.iloc[2:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_3)
mae_timexer_3 = mean_absolute_error(data_test.iloc[2:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_3)
mape_timexer_3 = mean_absolute_percentage_error(data_test.iloc[2:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_3)
r2_timexer_3 = r2_score(data_test.iloc[2:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_3)
print(f"For 3 day future -> RMSE: {rsme_timexer_3}, MAE: {mae_timexer_3}, MAPE: {mape_timexer_3}, R2: {r2_timexer_3}")

For 3 day future -> RMSE: 1.9298102487930722, MAE: 1.4700747576905278, MAPE: 0.06132653564103423, R2: 0.797709167219709


In [18]:
rsme_timexer_4 = root_mean_squared_error(data_test.iloc[3:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_4)
mae_timexer_4 = mean_absolute_error(data_test.iloc[3:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_4)
mape_timexer_4 = mean_absolute_percentage_error(data_test.iloc[3:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_4)
r2_timexer_4 = r2_score(data_test.iloc[3:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_4)
print(f"For 4 day future -> RMSE: {rsme_timexer_4}, MAE: {mae_timexer_4}, MAPE: {mape_timexer_4}, R2: {r2_timexer_4}")

For 4 day future -> RMSE: 2.087829022173322, MAE: 1.5927640319176994, MAPE: 0.06620172349305284, R2: 0.7627168847173842


In [21]:
rsme_timexer_5 = root_mean_squared_error(data_test.iloc[4:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_5)
mae_timexer_5 = mean_absolute_error(data_test.iloc[4:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_5)
mape_timexer_5 = mean_absolute_percentage_error(data_test.iloc[4:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_5)
r2_timexer_5 = r2_score(data_test.iloc[4:][['temperature_2m_max (°C)', 'temperature_2m_min (°C)']], timexer_pred_5)
print(f"For 5 day future -> RMSE: {rsme_timexer_5}, MAE: {mae_timexer_5}, MAPE: {mape_timexer_5}, R2: {r2_timexer_5}")

For 5 day future -> RMSE: 2.177988007969941, MAE: 1.667702322220879, MAPE: 0.06959593062215744, R2: 0.7413860207959644
